# 开源人物社会网格构建

组员：李焕貌   陈 彬   阮子昌   
指导老师：陈卫

## 1 Background INFO

### Six Degrees of Separation

Six degrees of separation is the idea that all people are six or fewer social connections away from each other. As a result, a chain of "friend of a friend" statements can be made to connect any two people in a maximum of six steps. 
<img align="float:right" src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Six_degrees_social_network.png/1024px-Six_degrees_social_network.png"/>

### Graph Database

A graph database (GDB) is a database that uses graph structures for semantic queries with nodes, edges, and properties to represent and store data.
<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/3/3a/GraphDatabase_PropertyGraph.png" height="50%" width="50%"/>
<center/>

- Nodes represent entities or instances
- Edges, aka relationships, are the lines that connect nodes to other nodes
- Properties are information associated to nodes

## 2 On Practice

### Data Source

- [Wikipedia The Free Ecyclopedia](https://en.wikipedia.org/w/index.php?title=List_of_presidents_of_the_United_States)

Data comes from the `infobox` of `Personal Details` on personal page.   
However, not all data is displayed on the Wikipedia.
For example, [John Adams - 2nd President](https://en.wikipedia.org/wiki/John_Adams)

### Data Object Modeling

Node Modeling
- Person Node
    - Person Name
    - Wikipedia Link
    - Is_President
- Family Node
    - Family Name
    - Wikipedia Link

```python
person_node = []
person_node.append({
    "name": person_name,
    "is_president": is_president,
    "wikipedia_url": person_url,
})
family_node = []
family_node.append({
    "name": family_name.lower(),
    "url": family_url
})
```

### Data Object Modeling

Node Modeling
```python
family_node.append({
    "name": family_name.lower(),
    "url": family_url
})
```

Why using `lower()` to `family_name`?   
There may be capitalization issues.
- [John F. Kennedy](https://en.wikipedia.org/wiki/John_F._Kennedy)
- [Edwin Schlossberg](https://en.wikipedia.org/wiki/Edwin_Schlossberg)

### Data Object Modeling

Relationship Modeling
- Marriage_between
- Child_Of
- Parent_Of
- Member_Of

```python
XXX_XX.append({
    "source": XXXXX_name,
    "target": XXXXX_name
})
```

### Data Object Modeling

```python
neo4j_data = {
    "PersonNode": person_node,
    "FamilyNode": family_node,
    "Marriage": marriage_between,
    "Parents": child_of,
    "Children": parent_of,
    "Family": member_of,
}
```

```json
{
    "PersonNode": [
        {
            "name": "George Washington",
            "is_president": true,
            "wikipedia_url": "https://..."
        }
    ],
    "FamilyNode": [
        {
            "name": "john custis (councillor)",
            "url": "https://..."
        }
    ]
}
```

### Programing - Web Crawler

All of the data structures were mentioned earlier, except the `node_set`.
```python
node_set = set()
if person_name in node_set:
    if is_president:
        for item in person_node:
            if item["name"] == person_name:
                item["is_president"] = True
    return
```

`node_set` is designed to not repeatedly visit someone's personal page.   
The parameter defaults to `False` when we call the function.

### Programing - Web Crawler

Core function
```python
get_person_info(person_url, person_name, is_president=False, depth=0)
```

- Whenever an accessible link is encountered, this funciton is called
- When `depth > MAX_DEPTH`, this function will return without running

### Programing - Web Crawler

Overall Logic of The Program
```python
for president_links in presidents_links:
    links = president_links.find_all("a")
    for link in links:
        url = "https://en.wikipedia.org" + link.get("href")
        name = link.get("title").replace("\u200b", "").replace("\n", "").strip()
        get_person_info(url, name, True, 0)

with open("data.json", "w") as f:
    json.dump(neo4j_data, f, indent=4)

```
The Unicode `\u200b` is zero-width space, which is used to mark a potential line break.

### Programing - Import Data

Neo4j is a graph database management system developed by Neo4j, Inc.
<center>
    <img align="middle" src="https://dist.neo4j.com/wp-content/uploads/20210423072428/neo4j-logo-2020-1.svg" height="50%" width="50%"/>
<center/>

Cypher is Neo4j’s graph query language that lets you retrieve data from the graph. It is like SQL, but for graphs.
```SQL
MATCH (nicole:Actor {name: 'Nicole Kidman'})-[:ACTED_IN]->(movie:Movie)
WHERE movie.year < $yearParameter
RETURN movie
```

### Programing - Import Data

Node Import
```SQL
CALL apoc.load.json("file:///data.json") YIELD value AS data
CALL apoc.periodic.iterate("
    UNWIND $data.PersonNode AS person
    RETURN person
", "
    MERGE (p:Person { name: person.name })
    SET p.is_president = person.is_president, p.wikipedia_url = person.wikipedia_url
", {batchSize: 500, params: {data: data}}) YIELD batches, total RETURN batches, total;
```

### Programing - Import Data

Relationship Import
```SQL
CALL apoc.load.json("file:///data.json") YIELD value AS data
CALL apoc.periodic.iterate("
    UNWIND $data.Children AS children
    RETURN children
", "
    MATCH (source:Person { name: children.source })
    MATCH (target:Person { name: children.target })
    MERGE (source)-[:parent_of]->(target)
", {batchSize: 500, params: {data: data}}) YIELD batches, total RETURN batches, total;
```

### Programing - Visualization

Using Matplotlib and NetworkX to visualize the path between.
<figure class="hald">
    <img src="https://matplotlib.org/_static/logo_light.svg" height="50%" wighth="50%"/>
    <img src="https://networkx.org/_static/networkx_logo.svg"height="50%" wighth="50%"/>
<figure/>

### Programing - Visualization

`shortestPath()` is a shorest path algorithm in neo4j.

```SQL
MATCH (p1:Person {name: $name1}), (p2:Person {name: $name2}),
    path = shortestPath((p1)-[*]-(p2))
RETURN path
```

### Programing - Visualization

A Driver object holds the details required to establish connections with a Neo4j database.
```python
driver = GraphDatabase.driver(uri, auth=(user, password))
```
A transaction is a unit of work that is either committed in its entirety or rolled back on failure.
```python
with driver.session() as session:
    path = session.execute_read(transaction_function, parma1, ...)
```

### Programing - Visualization

Put the `shortestPath()` into a python function.
```python

def shortest_path(tx, name1, name2):
    result = tx.run(
        "MATCH (p1:Person {name: $name1}), (p2:Person {name: $name2}),"
        "  path = shortestPath((p1)-[*]-(p2)) "
        "RETURN path",
        name1=name1,
        name2=name2
    )
    record = result.single()
    if record is None or record["path"] is None:
        raise ValueError(f"No path found between {name1} and {name2}")
    return record["path"]
```

### Programing - Visualization

```python
G = nx.DiGraph()
for i, rel in enumerate(path.relationships):
    start_node = rel.start_node
    end_node = rel.end_node
    G.add_edge(
        start_node["name"], end_node["name"],
        label=str(i),
        type=rel.type
    )
```
Create a NetworkX directed graph `G` and iterates through the relationships in the path.   
For each relationship, it adds an edge to the graph between the start and end nodes,along with the index of the relationship and its type as edge attributes.

### Programing - Visualization

Use the NetworkX and matplotlib to visualize the graph `G`.

```python
pos = nx.circular_layout(G)
edge_labels = nx.get_edge_attributes(G, "label")
edge_types = nx.get_edge_attributes(G, "type")
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_types, font_color="red")
nx.draw_networkx_labels(G, pos)
plt.axis("off")
plt.show()
```

## 3 Summary

- Drawback
    - Insufficietn Data
    - Confusing Entries
- Further
    - A more powerful database
    - ...